## Imports, function definitions

Note: DeepMoD does not work for python3.9

In [1]:
# pip install pysindy

In [2]:
# pip install DeePyMoD

In [3]:
# pip install --upgrade tensorflow

In [4]:
# pip install tensorflow-io

In [5]:
results_path = ""

In [6]:
import matplotlib.pyplot as plt

# General imports
import numpy as np
import torch
import pandas as pd

# DeePyMoD imports
from deepymod import DeepMoD
from deepymod.data import Dataset, get_train_test_loader
from deepymod.data.samples import Subsample_random
from deepymod.data.burgers import burgers_delta
from deepymod.model.constraint import LeastSquares
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.sparse_estimators import Threshold
from deepymod.training import train
from deepymod.training.sparsity_scheduler import Periodic, TrainTest, TrainTestPeriodic

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


import pickle
from sklearn.metrics import mean_squared_error
def load_data(X,y):
    return torch.from_numpy(X).float(),torch.from_numpy(y).float()

def uxt_2D_to1D(u,x,t):
    '''
    Function for reshaping u,x and t
    '''
    t_reg=[]
    x_reg=[]
    for i in range(len(t)):
        for j in range(len(x)):
            t_reg.append(t[i])
            x_reg.append(x[j])
    u_reg = u.reshape((u.size, 1))

    return u_reg,x_reg,t_reg

descr = ['',
 'u_{x}',
 'u_{xx}',
 'u_{xxx}',
 'u',
 'uu_{x}',
 'uu_{xx}',
 'uu_{xxx}',
 'u^2',
 'u^2u_{x}',
 'u^2u_{xx}',
 'u^2u_{xxx}']

def return_pde(w, rhs_description=descr, ut = 'u_t'):
    pde = ut + ' = '
    first = True
    for i in range(len(w)):
        if w[i] != 0:
            if not first:
                pde = pde + ' + '
            pde = pde + "(%05f)" % (w[i].real) + rhs_description[i] 
            first = False
    return pde


cuda


In [7]:
def error_metric(true_coeff,pred_coeff):
    wrmse = mean_squared_error(true_coeff,pred_coeff,squared=False)
    return wrmse

## Testing 3.1

In [8]:
# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(21,23):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  20
Dataset is using device:  cuda


2023-04-28 11:03:39.466273: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 11:03:40.039522: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 19750  MSE: 1.02e+05  Reg: 1.33e-17  L1: 4.13e+04 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (30634.085938)u_{xx} + (-1827.394531)uu_{xx}  rmse =  8858.975682867707
Dataset is using device:  cuda
 19750  MSE: 6.20e+04  Reg: 1.28e-11  L1: 8.61e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (1.236627) + (1393556.250000)u_{x} + (1264346.125000)u_{xx} + (614537.062500)u_{xxx} + (0.057855)u + (-5208.639648)uu_{x} + (10332.083984)uu_{xx} + (-9311.365234)uu_{xxx} + (-0.000786)u^2 + (-111.404472)u^2u_{x} + (-266.757507)u^2u_{xx} + (28.052864)u^2u_{xxx}  rmse =  571434.3236808357
Dataset is using device:  cuda
 19750  MSE: 9.06e+04  Reg: 6.00e-12  L1: 1.76e+07 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (13.289065) + (-292609.281250)u_{x} + (-37221.570312)u_{xx} + (5113.871094)u_{xxx} + (-0.727044)u + (14905.027344)uu_{x} + (-2445.635498)uu_{xx} + (-501.417236)uu_{xxx} + (0.009876)u^2 + (-184.763687)u^2u_{x} + (105.105453)u^2u

KeyboardInterrupt: 

In [8]:
# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(22,23):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  10
Dataset is using device:  cuda


2023-04-30 12:08:37.133991: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 12:08:37.898838: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  2250  MSE: 1.08e+05  Reg: 2.85e-05  L1: 6.00e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (94566.250000)u_{xx} + (3824.922852)u_{xxx} + (79083.203125)uu_{x} + (30704.273438)uu_{xx} + (-479.958984)uu_{xxx} + (-9329.969727)u^2u_{x} + (-4940.302734)u^2u_{xx} + (1.567871)u^2u_{xxx}  rmse =  36817.07938936757
Dataset is using device:  cuda
  1250  MSE: 1.08e+05  Reg: 1.47e-06  L1: 5.55e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000000) + (474607.343750)u_{xx} + (80477.296875)uu_{x} + (-3323.557129)uu_{xx} + (-8670.604492)u^2u_{x} + (-5151.579102)u^2u_{xx}  rmse =  138996.80687370893
Dataset is using device:  cuda
 19750  MSE: 1.06e+05  Reg: 7.51e-03  L1: 1.66e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000008) + (-2321.343750)u_{xx} + (3126.875000)u_{xxx} + (-10917.508789)uu_{x} + (-12786.554688)uu_{xx} + (-3561.617188)uu_{xxx} + (1361.524902)u^2u_{x} + (1640.197266)u^2u_{xx} + (393.595215)u^2u_{xxx}  rmse

## Testing 3.2

In [9]:
# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(23,25):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  8
Dataset is using device:  cuda
  2250  MSE: 1.08e+05  Reg: 6.38e-07  L1: 1.42e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.000001) + (-137672.906250)u_{x} + (-43870.902344)u_{xx} + (14220.280273)uu_{x} + (5317.693359)uu_{xx} + (299.685852)u^2u_{x}  rmse =  41941.509266277324
Dataset is using device:  cuda
  1500  MSE: 1.07e+05  Reg: 6.81e-13  L1: 1.06e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000000) + (-0.450739)u_{xx} + (0.045551)u_{xxx}  rmse =  0.17391629188448535
Dataset is using device:  cuda
 19750  MSE: 1.07e+05  Reg: 1.11e-02  L1: 4.23e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000004) + (-9845.843750)u_{xx} + (-27504.441406)u_{xxx} + (-17887.119141)uu_{x} + (-12277.539062)uu_{xx} + (3998.530029)uu_{xxx} + (2044.579102)u^2u_{x} + (1554.952148)u^2u_{xx} + (-98.121674)u^2u_{xxx}  rmse =  10593.712509162755
Dataset is using device:  cuda
 19750  MSE: 1.07e+05  Reg: 2.03e-02  L1: 9.7

## Testing 3.3

In [10]:


# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(25,27):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  5
Dataset is using device:  cuda
  1750  MSE: 1.08e+05  Reg: 4.06e-10  L1: 1.07e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000011) + (-0.790900)u_{xx}  rmse =  0.27161436302005726
Dataset is using device:  cuda
  2250  MSE: 1.09e+05  Reg: 4.13e-04  L1: 5.82e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000017) + (-187930.312500)u_{xx} + (-1681.708130)uu_{x} + (49398.085938)uu_{xx} + (222.916718)u^2u_{x} + (-3240.096191)u^2u_{xx}  rmse =  56103.62858673461
Dataset is using device:  cuda
 19750  MSE: 1.06e+05  Reg: 3.01e-06  L1: 2.94e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000001) + (358267.062500)u_{xx} + (36906.097656)u_{xxx} + (62790.527344)uu_{x} + (-25861.003906)uu_{xx} + (780.561279)uu_{xxx} + (-6904.799316)u^2u_{x} + (-1493.958374)u^2u_{xx} + (-531.301636)u^2u_{xxx}  rmse =  105821.97401225427
Dataset is using device:  cuda
  2250  MSE: 1.08e+05  Reg: 1.81e-02  L1: 1.29e+06 Algorithm c

## Testing 3.4

In [11]:


# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(27,29):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  3
Dataset is using device:  cuda
  1750  MSE: 1.07e+05  Reg: 8.43e-14  L1: 9.79e-01 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.118960)u_{xx} + (0.022755)u_{xxx}  rmse =  0.07791947629096038
Dataset is using device:  cuda
  2250  MSE: 1.08e+05  Reg: 7.87e-04  L1: 2.96e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000003) + (-291975.812500)u_{xx} + (11355.246094)uu_{x} + (62513.960938)uu_{xx} + (-1083.674683)u^2u_{x} + (-3320.419189)u^2u_{xx}  rmse =  86264.66017348286
Dataset is using device:  cuda
  2250  MSE: 1.08e+05  Reg: 8.32e-04  L1: 3.75e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000001) + (-401646.531250)u_{x} + (176890.250000)u_{xx} + (20911.509766)u_{xxx} + (161883.265625)uu_{x} + (3158.233887)uu_{xx} + (1429.360229)uu_{xxx} + (-13985.262695)u^2u_{x} + (-3200.667480)u^2u_{xx} + (-504.228302)u^2u_{xxx}  rmse =  135237.94773052365
Dataset is using device:  cuda
  6000  MSE: 1.08e+05  Reg: 

## Testing 3.5

In [8]:


# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(29,31):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  1
Dataset is using device:  cuda


2023-05-05 18:04:09.342978: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 18:04:09.983153: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  2250  MSE: 1.11e+05  Reg: 1.30e-11  L1: 3.11e+04 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (2340.268066)u_{xx} + (-283.823944)uu_{xx}  rmse =  680.484422792613
Dataset is using device:  cuda
 19750  MSE: 1.15e+05  Reg: 7.57e-04  L1: 5.47e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000030) + (17066.500000)u_{xx} + (-1927.220581)uu_{x} + (-4871.041992)uu_{xx} + (247.177292)u^2u_{x} + (343.060425)u^2u_{xx}  rmse =  5154.935488190566
Dataset is using device:  cuda
  2750  MSE: 1.12e+05  Reg: 1.42e-05  L1: 1.69e+07 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (283362.968750)u_{xx} + (186972.656250)u_{xxx} + (-65371.726562)uu_{xx} + (-42287.394531)uu_{xxx} + (3760.726562)u^2u_{xx} + (2370.115234)u^2u_{xxx}  rmse =  100554.59052824324
Dataset is using device:  cuda
  2250  MSE: 1.12e+05  Reg: 5.57e-10  L1: 1.47e+04 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.084437)u_{x} + (1568.421875)u_{xx} 